In [ ]:
import json
import sqlite3
import requests
import numpy as np
import pandas as pd
from pathlib import Path

BASE_URL = "https://api.open-meteo.com/v1/forecast"
DB_PATH = Path("../Data/DB/AERO.db")

In [ ]:
latlong = {"London":[51.50, -0.12],"New York": [40.71, -74.00],"Tokyo": [35.67, 139.65],"Sydney": [-33.86, 151.20],"Reykjavik": [64.14, -21.89]}
cities = ["London", "New York", "Tokyo", "Sydney", "Reykjavik"]
data1 = []

for lat, long in latlong.values():
    resp = requests.get(f"{BASE_URL}?latitude={lat}&longitude={long}&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m&forecast_days=3").json()
    resp["city"] = [n for n, l in latlong.items() if l == [lat, long]][0]
    data1.append(resp)

data1

[{'latitude': 51.5,
  'longitude': -0.120000124,
  'generationtime_ms': 0.09393692016601562,
  'utc_offset_seconds': 0,
  'timezone': 'GMT',
  'timezone_abbreviation': 'GMT',
  'elevation': 1.0,
  'hourly_units': {'time': 'iso8601',
   'temperature_2m': '°C',
   'relative_humidity_2m': '%',
   'wind_speed_10m': 'km/h'},
  'hourly': {'time': ['2025-12-13T00:00',
    '2025-12-13T01:00',
    '2025-12-13T02:00',
    '2025-12-13T03:00',
    '2025-12-13T04:00',
    '2025-12-13T05:00',
    '2025-12-13T06:00',
    '2025-12-13T07:00',
    '2025-12-13T08:00',
    '2025-12-13T09:00',
    '2025-12-13T10:00',
    '2025-12-13T11:00',
    '2025-12-13T12:00',
    '2025-12-13T13:00',
    '2025-12-13T14:00',
    '2025-12-13T15:00',
    '2025-12-13T16:00',
    '2025-12-13T17:00',
    '2025-12-13T18:00',
    '2025-12-13T19:00',
    '2025-12-13T20:00',
    '2025-12-13T21:00',
    '2025-12-13T22:00',
    '2025-12-13T23:00',
    '2025-12-14T00:00',
    '2025-12-14T01:00',
    '2025-12-14T02:00',
    '2025-12

In [16]:
print(len(data1[0]['hourly']['time'])==data1[0]['hourly']['relative_humidity_2m'])

False


In [23]:
rows = []

for di in data1:
    city = di["city"]
    hourly = di["hourly"]

    times = hourly["time"]
    temps = hourly["temperature_2m"]
    hums  = hourly["relative_humidity_2m"]
    winds = hourly["wind_speed_10m"]

    for i in range(len(times)):
        rows.append({
            "city": city,
            "forecast_time": times[i],
            "temp_c": temps[i],
            "humidity": hums[i],
            "wind_kph": winds[i]
        })

rows

[{'city': 'London',
  'forecast_time': '2025-12-13T00:00',
  'temp_c': 7.5,
  'humidity': 91,
  'wind_kph': 7.1},
 {'city': 'London',
  'forecast_time': '2025-12-13T01:00',
  'temp_c': 7.1,
  'humidity': 91,
  'wind_kph': 5.8},
 {'city': 'London',
  'forecast_time': '2025-12-13T02:00',
  'temp_c': 6.7,
  'humidity': 91,
  'wind_kph': 5.7},
 {'city': 'London',
  'forecast_time': '2025-12-13T03:00',
  'temp_c': 6.4,
  'humidity': 92,
  'wind_kph': 6.0},
 {'city': 'London',
  'forecast_time': '2025-12-13T04:00',
  'temp_c': 6.1,
  'humidity': 94,
  'wind_kph': 6.0},
 {'city': 'London',
  'forecast_time': '2025-12-13T05:00',
  'temp_c': 5.8,
  'humidity': 94,
  'wind_kph': 6.0},
 {'city': 'London',
  'forecast_time': '2025-12-13T06:00',
  'temp_c': 5.6,
  'humidity': 96,
  'wind_kph': 5.4},
 {'city': 'London',
  'forecast_time': '2025-12-13T07:00',
  'temp_c': 5.3,
  'humidity': 97,
  'wind_kph': 5.0},
 {'city': 'London',
  'forecast_time': '2025-12-13T08:00',
  'temp_c': 5.0,
  'humidity'

In [31]:
conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()
cur.execute(
    """
    DROP TABLE IF EXISTS forecasts;
    """
)

cur.execute(
    """
    CREATE TABLE IF NOT EXISTS forecasts(
    id	INTEGER PRIMARY KEY AUTOINCREMENT,
    city TEXT,
    forecast_time TEXT,
    temp_c REAL,
    humidity INTEGER,
    wind_kph REAL
    );
    """
)


com_for_insert = """
    INSERT OR REPLACE INTO forecasts(city, forecast_time, temp_c, humidity, wind_kph)
    VALUES(:city, :forecast_time, :temp_c, :humidity, :wind_kph)
    """

for row in rows:
    cur.execute(com_for_insert, row)

conn.commit()
conn.close()

In [32]:
conn = sqlite3.connect(DB_PATH)
df = pd.read_sql("""SELECT * FROM forecasts""", conn)
conn.close()

In [33]:
df

,id,city,forecast_time,temp_c,humidity,wind_kph
0,1,London,2025-12-13T00:00,7.5,91,7.1
1,2,London,2025-12-13T01:00,7.1,91,5.8
2,3,London,2025-12-13T02:00,6.7,91,5.7
3,4,London,2025-12-13T03:00,6.4,92,6.0
4,5,London,2025-12-13T04:00,6.1,94,6.0
...,...,...,...,...,...,...
355,356,Reykjavik,2025-12-15T19:00,-5.3,76,2.3
356,357,Reykjavik,2025-12-15T20:00,-5.4,79,2.2
357,358,Reykjavik,2025-12-15T21:00,-5.5,78,3.1
358,359,Reykjavik,2025-12-15T22:00,-5.8,78,3.3


In [62]:
groupt = df.groupby("city")["temp_c"].max()

h_temp = groupt.max()
h_temp_city = groupt.idxmax()

print(f"Highest predicted tempreture in next 3 days has {h_temp_city} with tempreture of {h_temp:.2f}")

Highest predicted tempreture in next 3 days has Sydney with tempreture of 32.50


In [61]:
groupws = df.groupby("city")["wind_kph"].mean()

h_ws = groupws.max()
h_ws_city = groupws.idxmax()

print(f"Highest average wind speed in next 3 days has {h_ws_city} with speed of {h_ws:.2f}")

Highest average wind speed in next 3 days has New York with speed of 15.13


In [69]:
groupht = groupt = df.groupby("city")["temp_c"].max()
grouplt = groupt = df.groupby("city")["temp_c"].min()

temp_swing = {}

for city in cities:
    swing = groupht[city]-grouplt[city]
    temp_swing[city] = swing

h_ts = max(temp_swing.values())
h_ts_city = [city for city, swg in temp_swing.items() if swg == h_ts][0]

print(f"Highest tempreture swing in next 3 days has {h_ts_city} with speed of {h_ts:.2f}")

Highest tempreture swing in next 3 days has Sydney with speed of 14.40


In [71]:
groupat = df.groupby("city")["temp_c"].mean()

for city in cities:
    avg_temp = groupat[city]
    print(f"Average predicted tempreture in {city} is {avg_temp:.2f}")

Average predicted tempreture in London is 8.77
Average predicted tempreture in New York is -2.52
Average predicted tempreture in Tokyo is 6.34
Average predicted tempreture in Sydney is 22.42
Average predicted tempreture in Reykjavik is 0.52


In [87]:
df["fog_rain_risk"] = np.ones(360)

In [88]:
df

,id,city,forecast_time,temp_c,humidity,wind_kph,fog_rain_risk
0,1,London,2025-12-13T00:00,7.5,91,7.1,1.0
1,2,London,2025-12-13T01:00,7.1,91,5.8,1.0
2,3,London,2025-12-13T02:00,6.7,91,5.7,1.0
3,4,London,2025-12-13T03:00,6.4,92,6.0,1.0
4,5,London,2025-12-13T04:00,6.1,94,6.0,1.0
...,...,...,...,...,...,...,...
355,356,Reykjavik,2025-12-15T19:00,-5.3,76,2.3,1.0
356,357,Reykjavik,2025-12-15T20:00,-5.4,79,2.2,1.0
357,358,Reykjavik,2025-12-15T21:00,-5.5,78,3.1,1.0
358,359,Reykjavik,2025-12-15T22:00,-5.8,78,3.3,1.0


In [89]:
df["fog_rain_risk"] = df["fog_rain_risk"].where(df["humidity"]>90, 0)

In [90]:
df

,id,city,forecast_time,temp_c,humidity,wind_kph,fog_rain_risk
0,1,London,2025-12-13T00:00,7.5,91,7.1,1.0
1,2,London,2025-12-13T01:00,7.1,91,5.8,1.0
2,3,London,2025-12-13T02:00,6.7,91,5.7,1.0
3,4,London,2025-12-13T03:00,6.4,92,6.0,1.0
4,5,London,2025-12-13T04:00,6.1,94,6.0,1.0
...,...,...,...,...,...,...,...
355,356,Reykjavik,2025-12-15T19:00,-5.3,76,2.3,0.0
356,357,Reykjavik,2025-12-15T20:00,-5.4,79,2.2,0.0
357,358,Reykjavik,2025-12-15T21:00,-5.5,78,3.1,0.0
358,359,Reykjavik,2025-12-15T22:00,-5.8,78,3.3,0.0


In [ ]:
print(f"Count of rows with 90+% humidity is {df["fog_rain_risk"].sum()}")
df = df.drop("fog_rain_risk", axis=1)

Count of rows with 90+% humidity is 42.0


In [ ]:
df

KeyError: "['fog_rain_risk'] not found in axis"